In [ ]:
import sys
import os
import numpy
import scipy
from matplotlib import pyplot as plt

In [ ]:
numpy.set_printoptions(threshold=sys.maxsize)

In [ ]:
%matplotlib inline

In [ ]:
%run analysis.py

In [ ]:
pol = 'ee'
freq_channel = 300
time_integration = 0
bad_ants=[0, 2, 11, 24, 50, 53, 54, 67, 69, 98,122, 136, 139]

In [ ]:
mdm_fn = "/Users/matyasmolnar/Downloads/HERA_Data/robust_cal/zen.2458098.43869.HH.uvh5"
if os.path.exists(mdm_fn):
    filename = mdm_fn
else:
    filename = "./zen.2458098.43869.HH.uvh5"

In [ ]:
hdraw = HERAData(filename)
reds = get_reds(hdraw.antpos, pols=[pol])
data, flags, nsamples = hdraw.read(freq_chans=[freq_channel])

In [ ]:
flt_bls = fltBad(reds, bad_ants)
cRedG=groupBls(flt_bls)
# This groups reundant baselines. Format is [[groupid, i, j]]

In [ ]:
# Collect the data together
cData=numpy.hstack([data[(zz[1], zz[2], "ee")] for zz in cRedG])
print(cData.shape)

In [ ]:
bl_id_seperations = numpy.unique(cRedG[:,0], return_index=True)[1][1:]

fig, ax = plt.subplots(figsize=(13,4))
ax.matshow(np.absolute(cData), aspect='auto')
for bl_id_seperation in bl_id_seperations:
    plt.axvline(x=bl_id_seperation, color='white', linestyle='-.', linewidth=1)
ax.grid(False)
ax.set_xlabel('Baseline ID')
ax.set_ylabel('Time Integration')
plt.show()

### Relative calibration step

In [ ]:
# Finding all the antennas used in our flagged data
ants = numpy.unique(cRedG[:,1:])
no_unq_bls = numpy.unique(cRedG[:, 0]).size

In [ ]:
# Setup initial parameters
xvis=numpy.ones(no_unq_bls*2) # Number of unique baselines; complex vis
xgains=numpy.ones(ants.size*2) # Complex gain
rel_xparams = numpy.hstack([xvis, xgains])

In [ ]:
# This creates a compiled version of likelihood where the first (the redundant groups),
# second (the error distribution) and third (observed visibilities) parameters 
# have already been filled in [-- this is "partial" fn application]
ff=jit(functools.partial(relative_logLkl, relabelAnts(cRedG), "cauchy", \
                         cData[time_integration, :]))
res_rel=scipy.optimize.minimize(ff, rel_xparams, jac=jacrev(ff))

In [ ]:
def split_results(results):
    ''' Split results from minimization into visibility and gains arrays '''
    vis_params, gains_params = numpy.split(results['x'], [no_unq_bls*2,])
    vis_params = vis_params.reshape((-1, 2))
    gains_params = gains_params.reshape((-1, 2))
    res_vis = vis_params[:, 0] + 1j*vis_params[:, 1]
    res_gains = gains_params[:, 0] + 1j*gains_params[:, 1]
    return res_vis, res_gains

In [ ]:
res_rel_vis, res_rel_gains = split_results(res_rel)

### Solving for degeneracies

In [ ]:
# Setup initial parameters
xamp = 1.
xoverall_phase = 1.
xphase_grad_x = 1.
xphase_grad_y = 1.
deg_xparams = numpy.hstack([xgains, xamp, xoverall_phase, xphase_grad_x, xphase_grad_y])

In [ ]:
ref_ant = 12 # reference antenna number
ref_ant_idx = condenseMap(ants)[ref_ant]

# constraints
def avg_amp(x):
    """Constraint that average of gain amplitudes must be equal to 1"""
    rel_gains_comps = x[:ants.size*2].reshape((-1, 2))
    rel_gains = rel_gains_comps[:, 0] + 1j*rel_gains_comps[:, 1]
    return np.average(np.abs(rel_gains)) - 1

# def positive_amp(x):
#     """Constraint that gain amplitudes must be positive"""
#     rel_gains_comps = x[:ants.size*2].reshape((-1, 2))
#     rel_gains = rel_gains_comps[:, 0] + 1j*rel_gains_comps[:, 1]
#     return np.abs(rel_gains)

def avg_phase(x):
    """Constraint that average of gain phases must be equal to 0"""
    rel_gains_comps = x[:ants.size*2].reshape((-1, 2))
    rel_gains = rel_gains_comps[:, 0] + 1j*rel_gains_comps[:, 1]
#     Do we need to do a different kind of averaging? https://en.wikipedia.org/wiki/Mean_of_circular_quantities
    return scipy.stats.circmean(np.angle(rel_gains))

def ref_phase(x):
    """Set argument of reference antenna to zero to set overall phase"""
    rel_gains_comps = x[:ants.size*2].reshape((-1, 2))
    rel_gains = rel_gains_comps[:, 0] + 1j*rel_gains_comps[:, 1]
    return np.angle(rel_gains[ref_ant_idx])

# Not sure this last constraint is correct...
def phase_grad(x):
    """Constraint that phase gradient is zero"""
    rel_gains_comps, deg_params = np.split(x, [2*ants.size,])
    _, overall_phase, phase_grad_x, phase_grad_y = deg_params
    rel_gains_comps = rel_gains_comps.reshape((-1, 2))
    rel_gains = rel_gains_comps[:, 0] + 1j*rel_gains_comps[:, 1]
    x_ant_ref_pos, y_ant_ref_pos = hdraw.antpos[ref_ant][:2]
    return np.angle(rel_gains[ref_ant_idx]) - overall_phase - x_ant_ref_pos * phase_grad_x - \
           y_ant_ref_pos * phase_grad_y
    
cons = [{'type': 'eq', 'fun': avg_amp},
#         {'type': 'ineq', 'fun': positive_amp},
        {'type': 'eq', 'fun': avg_phase},
        {'type': 'eq', 'fun': ref_phase},
        {'type': 'eq', 'fun': phase_grad}]

In [ ]:
ff=jit(functools.partial(optimal_logLkl, relabelAnts(cRedG), "gaussian", \
                         cData[time_integration, :], red_ant_sep(cRedG, hdraw.antpos), \
                         reformatCArray(res_rel_vis)))
res_deg=scipy.optimize.minimize(ff, deg_xparams, constraints=cons, jac=jacrev(ff), method='trust-constr')

In [ ]:
res_deg['success']

In [ ]:
new_gain_params, deg_params = np.split(res_deg['x'], [ants.size*2,])
new_gain_params = new_gain_params.reshape((-1, 2))
new_gains = new_gain_params[:,0] + 1j*new_gain_params[:,1]

In [ ]:
print(deg_params, np.average(np.abs(new_gains)), np.average(np.angle(new_gains)))